In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last" #"last expr -> all로 바꾸면 전체가 나온다. "
pd.options.display.max_columns = 200
pd.options.display.max_rows = 250
pd.options.display.max_colwidth = 100

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_finance import candlestick_ohlc
mpl.rc('font', family='NanumGothic') #폰트 한글 포함 가능 설정
mpl.rc('axes', unicode_minus=False) #유니코드에서 음수 부호 설정

from sklearn.preprocessing import MinMaxScaler, RobustScaler

from keras.optimizers import RMSprop,adam
from keras.callbacks import History
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)

Using TensorFlow backend.
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user/anaconda3/envs/park/l

### AUC CI interval, PR confidence interval

In [76]:
rng=np.random.RandomState(1234)

def get_ci_auc( y_true, y_pred ): 

    from scipy.stats import sem
    from sklearn.metrics import roc_auc_score 

    n_bootstraps = 1000   
    bootstrapped_scores = []   

    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.random_integers(0, len(y_pred) - 1, len(y_pred))

        if len(np.unique(y_true[indices])) < 2:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            continue

        score = roc_auc_score(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)   

    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

   # 90% c.i.
   # confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
   # confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
 
   # 95% c.i.
    confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
   
    return confidence_lower,confidence_upper

In [77]:
rng=np.random.RandomState(1234)

def get_ci_pr( y_true, y_pred ): 

    from scipy.stats import sem
    from sklearn.metrics import roc_auc_score 

    n_bootstraps = 1000   
    bootstrapped_scores = []   

    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.random_integers(0, len(y_pred) - 1, len(y_pred))

        if len(np.unique(y_true[indices])) < 2:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            continue

        score = average_precision_score(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)   

    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

   # 90% c.i.
   # confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
   # confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
 
   # 95% c.i.
    confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
   
    return confidence_lower,confidence_upper

In [2]:
pd_encoded=pd.read_csv("./backup1007/scaled_padded_191206.csv")

pd_encoded=pd_encoded.rename(columns={"104":'exam_cul',
                           "105":'code_name2',
                           "106":'result',
                           "107": 'exam_cul3',
                           "108": 'result3',
                           "109": 'G',
                           "110":'time',
                           "111":'censored'})

pd_encoded=pd_encoded.drop(["Unnamed: 0"], axis=1)

pd_encoded=pd_encoded.rename(columns={"0":'idx',
                           "1":'admday',
                           "2":'labday',
                           "3": 'time(round)'})

pd_encoded[['idx','admday','labday','time(round)']]=pd_encoded[['idx','admday','labday','time(round)']].astype(int)

outcome=pd.read_csv("./backup1007/pad_total_input191107.csv")
outcome=outcome.drop(["Unnamed: 0"], axis=1)

col=['result','result3','exam_cul','exam_cul3','censored']
for i in col:
    outcome.loc[outcome[i].isna(),i]=0
    outcome.loc[outcome[i]==True,i]=1

/home/user/anaconda3/envs/park/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (106,112) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (342,348) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
del pd_encoded

In [4]:
outcome['SIRS(PR)']=outcome['PR(mean)']>90
outcome['SIRS(RR)']=outcome['RR(mean)']>20
outcome.loc[(outcome['BT(min)']<36) |(outcome['BT(max)']>38),'SIRS(fever)']=True

outcome.loc[(outcome['WBC(min)']<4.0) | (outcome['WBC(max)']>12.0) | (outcome['IG %(mean)']>10),'SIRS(WBC)']=True

outcome[['SIRS(fever)','SIRS(WBC)']]=outcome[['SIRS(fever)','SIRS(WBC)']].fillna(False)

outcome['SIRS(score)']=outcome[['SIRS(PR)','SIRS(RR)','SIRS(fever)','SIRS(WBC)']].sum(axis=1)

outcome['SIRS_p']=outcome['SIRS(score)']>=2

outcome['SIRS_p']=outcome['SIRS_p'].replace(True,1)

In [5]:
outcome['MAP']=outcome['SBP(mean)']*1/3 + outcome['DBP(mean)']*2/3

In [6]:
outcome.loc[outcome['Platelet(mean)']>=150, 'sofa(plt)']=0
outcome.loc[(outcome['Platelet(mean)']<150) &(outcome['Platelet(mean)']>=100), 'sofa(plt)']=1
outcome.loc[(outcome['Platelet(mean)']<100) &(outcome['Platelet(mean)']>=50), 'sofa(plt)']=2
outcome.loc[(outcome['Platelet(mean)']<50) &(outcome['Platelet(mean)']>=20), 'sofa(plt)']=3
outcome.loc[(outcome['Platelet(mean)']<20), 'sofa(plt)']=4

outcome.loc[(outcome['Bilirubin,total(mean)']<1.2), 'sofa(bil)']=0
outcome.loc[(outcome['Bilirubin,total(mean)']<2.0) & (outcome['Bilirubin,total(mean)']>=1.2), 'sofa(bil)']=1
outcome.loc[(outcome['Bilirubin,total(mean)']<6) & (outcome['Bilirubin,total(mean)']>=2.0), 'sofa(bil)']=2
outcome.loc[(outcome['Bilirubin,total(mean)']<12) & (outcome['Bilirubin,total(mean)']>=6), 'sofa(bil)']=3
outcome.loc[(outcome['Bilirubin,total(mean)']>=12), 'sofa(bil)']=4

outcome['MAP']=outcome['SBP(mean)']*1/3 + outcome['DBP(mean)']*2/3
outcome.loc[outcome['MAP']>=70, 'sofa(cardio)']=0
outcome.loc[outcome['MAP']<70, 'sofa(cardio)']=1

outcome.loc[(outcome['Creatinine(mean)']<1.2), 'sofa(cr)']=0
outcome.loc[(outcome['Creatinine(mean)']<2.0) & (outcome['Creatinine(mean)']>=1.2), 'sofa(cr)']=1
outcome.loc[(outcome['Creatinine(mean)']<3.5) & (outcome['Creatinine(mean)']>=2.0), 'sofa(cr)']=2
outcome.loc[(outcome['Creatinine(mean)']<5) & (outcome['Creatinine(mean)']>=3.5), 'sofa(cr)']=3
outcome.loc[(outcome['Creatinine(mean)']>=5), 'sofa(cr)']=4

outcome['sofa(score)']=outcome[['sofa(plt)','sofa(bil)','sofa(cardio)','sofa(cr)']].sum(axis=1)
outcome['sofa_p']=outcome['sofa(score)']>=2

outcome['sofa_p']=outcome['sofa_p'].replace(True,1)

In [7]:
outcome.loc[outcome['RR(mean)']>=29, 'mews(RR)']=3
outcome.loc[(outcome['RR(mean)']<29) &(outcome['RR(mean)']>=21), 'mews(RR)']=2
outcome.loc[(outcome['RR(mean)']<21) &(outcome['RR(mean)']>=15), 'mews(RR)']=1
outcome.loc[(outcome['RR(mean)']<15) &(outcome['RR(mean)']>=9), 'mews(RR)']=0
outcome.loc[(outcome['RR(mean)']<9), 'mews(RR)']=2

outcome.loc[(outcome['PR(mean)']<=40), 'mews(PR)']=2
outcome.loc[(outcome['PR(mean)']<=50) & (outcome['PR(mean)']>40), 'mews(PR)']=1
outcome.loc[(outcome['PR(mean)']<=100) & (outcome['PR(mean)']>50), 'mews(PR)']=0
outcome.loc[(outcome['PR(mean)']<=110) & (outcome['PR(mean)']>100), 'mews(PR)']=1
outcome.loc[(outcome['PR(mean)']<=129) & (outcome['PR(mean)']>110), 'mews(PR)']=2
outcome.loc[(outcome['PR(mean)']>129), 'mews(PR)']=3

outcome.loc[(outcome['SBP(mean)']<=70), 'mews(SBP)']=3
outcome.loc[(outcome['SBP(mean)']<=80) & (outcome['SBP(mean)']>70), 'mews(SBP)']=2
outcome.loc[(outcome['SBP(mean)']<=100) & (outcome['SBP(mean)']>80), 'mews(SBP)']=1
outcome.loc[(outcome['SBP(mean)']<200) & (outcome['SBP(mean)']>100), 'mews(SBP)']=0
outcome.loc[(outcome['SBP(mean)']>200), 'mews(SBP)']=2

outcome.loc[(outcome['BT(mean)']<=35), 'mews(BT)']=2
outcome.loc[(outcome['BT(mean)']<=36) & (outcome['BT(mean)']>35), 'mews(BT)']=1
outcome.loc[(outcome['BT(mean)']<=38) & (outcome['BT(mean)']>36), 'mews(BT)']=0
outcome.loc[(outcome['BT(mean)']<=38.5) & (outcome['BT(mean)']>38), 'mews(BT)']=1
outcome.loc[(outcome['BT(mean)']>38.5), 'mews(BT)']=2


outcome['mews(score)']=outcome[['mews(RR)','mews(PR)','mews(SBP)','mews(BT)']].sum(axis=1)
outcome['mews_p']=outcome['mews(score)']>=4

outcome['mews_p']=outcome['mews_p'].replace(True,1)

In [8]:
sofa = pd.DataFrame(outcome.groupby(['idx','admday'])["sofa(score)"].agg(min))

In [9]:
outcome = outcome.merge(sofa, on=['idx','admday'],how='left')

In [10]:
outcome.loc[outcome['sofa(score)_y']>=2,"sofa_real"]=True
outcome['sofa_real']=outcome['sofa_real'].replace(True,1).fillna(0)

In [11]:
outcome[['result3','sofa_real']].sum(axis=0)

result3       2725.0
sofa_real    41707.0
dtype: float64

In [19]:
outcome['SIRS(score)'].unique()

array([0, 1, 3, 2, 4])

In [39]:
a=outcome['sofa(score)_x'].unique()
a.sort()
type(a[0])

numpy.float64

In [34]:
outcome['mews(score)'].unique()

array([ 2.,  3.,  1.,  5.,  4.,  0.,  6.,  7.,  8.,  9., 10.])

In [24]:
outcome['SIRS(score)']>

0          False
1          False
2          False
3          False
4          False
           ...  
1362860    False
1362861    False
1362862    False
1362863    False
1362864    False
Name: SIRS(score), Length: 1362865, dtype: bool

In [25]:
confusion = confusion_matrix(outcome['result3'],outcome['SIRS(score)']>1)

In [26]:
sensitivity = confusion[1,1]/(confusion[1,0]+confusion[1,1])
specificity = confusion[0,0]/(confusion[0,1]+confusion[0,0])
precicion = confusion[1,1]/(confusion[0,1]+confusion[1,1])


In [29]:
precicion

0.01134258009966359

In [12]:
from sklearn.metrics import confusion_matrix
name = ['sofa_real','SIRS_p','mews_p']
fusion_x = []
for i in name:
    confusion = confusion_matrix(outcome['result3'],outcome[i])

    sensitivity = confusion[1,1]/(confusion[1,0]+confusion[1,1])
    specificity = confusion[0,0]/(confusion[0,1]+confusion[0,0])
    precicion = confusion[1,1]/(confusion[0,1]+confusion[1,1])

    fusion_x.append({"threshold": i,
                     "sensitivity":sensitivity,
                     "specificity":specificity,
                    "precision":precicion})

In [13]:
pd.options.display.float_format = '{:.3f}'.format

table = pd.DataFrame(fusion_x)
table

,threshold,sensitivity,specificity,precision
0,sofa_real,0.088,0.970,0.006
1,SIRS_p,0.463,0.919,0.011
2,mews_p,0.116,0.993,0.031


In [60]:
name2= ['SIRS(score)','mews(score)','sofa(score)_x']
fusion_y = []
for i in name2:
    number = outcome[i].unique()
    number.sort()
    for j in range(len(number)):
        confusion = confusion_matrix(outcome['result3'],outcome[i]>=number[j])

        sensitivity = confusion[1,1]/(confusion[1,0]+confusion[1,1])
        specificity = confusion[0,0]/(confusion[0,1]+confusion[0,0])
        precicion = confusion[1,1]/(confusion[0,1]+confusion[1,1])

        fusion_y.append({"criteria":i,
                        "threshold":j,
                        "sensitivity":sensitivity,
                     "specificity":specificity,
                    "precision":precicion})

In [61]:
pd.options.display.float_format = '{:.3f}'.format

table = pd.DataFrame(fusion_y)
table.to_csv("./table.csv")

In [62]:
table

,criteria,threshold,sensitivity,specificity,precision
0,SIRS(score),0,1.000,0.000,0.002
1,SIRS(score),1,0.788,0.669,0.005
2,SIRS(score),2,0.463,0.919,0.011
3,SIRS(score),3,0.182,0.988,0.029
4,SIRS(score),4,0.040,0.999,0.051
5,mews(score),0,1.000,0.000,0.002
6,mews(score),1,0.995,0.004,0.002
7,mews(score),2,0.560,0.737,0.004
8,mews(score),3,0.266,0.960,0.013
9,mews(score),4,0.116,0.993,0.031


In [66]:
outcome['sofa(score)_x'].unique()

array([ 1.,  4.,  3.,  0.,  2.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [67]:
outcome['sirs_rescore']=outcome['SIRS(score)']/4
outcome['mews_rescore']=outcome['mews(score)']/10
outcome['sofa_rescore']=outcome['sofa(score)_x']/12

In [71]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [72]:
sirs_pr = average_precision_score(outcome['result3'],outcome['sirs_rescore'])
mews_pr = average_precision_score(outcome['result3'],outcome['mews_rescore'])
sofa_pr = average_precision_score(outcome['result3'],outcome['sofa_rescore'])


In [74]:
sirs_auc = roc_auc_score(outcome['result3'],outcome['sirs_rescore'])
mews_auc = roc_auc_score(outcome['result3'],outcome['mews_rescore'])
sofa_auc = roc_auc_score(outcome['result3'],outcome['sofa_rescore'])


In [73]:
sirs_pr, mews_pr, sofa_pr

(0.01133459018526074, 0.009650725934768001, 0.009774206277709804)

In [75]:
sirs_auc, mews_auc, sofa_auc

(0.7777779798167026, 0.6726900656340963, 0.7383297004099552)

In [78]:
lower_sirs, upper_sirs = get_ci_auc(outcome['result3'],outcome['sirs_rescore'])
lower_mews, upper_mews = get_ci_auc(outcome['result3'],outcome['mews_rescore'])
lower_sofa, upper_sofa = get_ci_auc(outcome['result3'],outcome['sofa_rescore'])

/home/user/anaconda3/envs/park/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 1362864 + 1) instead
  del sys.path[0]
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 1362864 + 1) instead
  del sys.path[0]
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 1362864 + 1) instead
  del sys.path[0]
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 1362864 + 1) instead
  del sys.path[0]
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 1362864 + 1) instead
  del sys.path[0]
/home/user/anaconda3

In [80]:
print(lower_sirs, upper_sirs)
print(lower_mews, upper_mews)
print(lower_sofa, upper_sofa)

0.7684558967752355 0.7866303871440712
0.6624596759150978 0.6826146488663976
0.7280863066656407 0.7476121007084


In [79]:
lower_sirs_pr, upper_sirs_pr = get_ci_pr(outcome['result3'],outcome['sirs_rescore'])
lower_mews_pr, upper_mews_pr = get_ci_pr(outcome['result3'],outcome['mews_rescore'])
lower_sofa_pr, upper_sofa_pr = get_ci_pr(outcome['result3'],outcome['sofa_rescore'])

/home/user/anaconda3/envs/park/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 1362864 + 1) instead
  del sys.path[0]


In [81]:
print(lower_sirs_pr, upper_sirs_pr)
print(lower_mews_pr, upper_mews_pr)
print(lower_sofa_pr, upper_sofa_pr)

0.010396082388642897 0.012677288195642985
0.008232615021112015 0.011223858860305361
0.008691450916725796 0.01120421910501063
